In [ ]:
!pip install --no-deps -r colab_multitfa.txt

In [ ]:
from operator import attrgetter

In [ ]:
import numpy as np
import pandas as pd
from cobra.io import load_model

In [ ]:
from multitfa.core import tmodel
from multitfa.analysis import variability_legacy_cplex

In [ ]:
cobra_model = load_model("e_coli_core")

Compartment specific pH and ionic strength information. Ionic strength units should be in M.

In [ ]:
pH_I_dict = {
    "pH": {"c": 7.5, "e": 7, "p": 7},
    "I": {"c": 0.25, "e": 0, "p": 0},
}

Membrane potentials between different compartments. Units for membrane potential is mV.

In [ ]:
del_psi_dict = {
    "c": {"c": 0, "e": 0, "p": 150},
    "e": {"c": 0, "e": 0, "p": 0},
    "p": {"c": -150, "e": 0, "p": 0},
}

In [ ]:
del_psi = pd.DataFrame.from_dict(data=del_psi_dict)
comp_info = pd.DataFrame.from_dict(data=pH_I_dict)

In [ ]:
cobra_model.boundary

In [ ]:
excl_reactions = [
    rxn.id
    for rxn in set(
        cobra_model.boundary
        + [
            cobra_model.reactions.BIOMASS_Ecoli_core_w_GAM,
            cobra_model.reactions.O2t,
            cobra_model.reactions.H2Ot,
        ]
    )
]

In [ ]:
tfa_model = tmodel(
    cobra_model,
    Exclude_list=excl_reactions,
    compartment_info=comp_info,
    membrane_potential=del_psi,
)

In [ ]:
for met in tfa_model.metabolites:
    met.Kegg_id = "bigg.metabolite:" + met.id[:-2]

tfa_model.update()

In [ ]:
print("Number of constraints before update {}\n".format(len(cobra_model.constraints)))
print("Number of reactions in the model {}\n".format(len(tfa_model.reactions)))
print(
    "Number of reactions excluded from thermodynamic analysis {}\n".format(
        len(tfa_model.Exclude_reactions)
    )
)
print("Number of mass balance constraints {}\n".format(len(tfa_model.metabolites)))
num_cons = 2 * 3 * (len(tfa_model.reactions) - len(tfa_model.Exclude_reactions)) + len(
    tfa_model.metabolites
)
print("Number of constraints to be present after update {}\n".format(num_cons))
print(
    "Number of constraints present after update {}\n".format(len(tfa_model.constraints))
)

In [ ]:
atp = tfa_model.metabolites.get_by_id("atp_c")

In [ ]:
print("Standard Gibbs energy of formation of ATP is {} KJ/mol".format(atp.delG_f))
print("initial concentrations")
print("{}\t{}\t{}".format(atp.id, atp.concentration_min, atp.concentration_max))

In [ ]:
atp.concentration_min = 2e-3
atp.concentration_max = 5e-2
tfa_model.update()

In [ ]:
print("Concentrations after modifying")
print("{}\t{}\t{}".format(atp.id, atp.concentration_min, atp.concentration_max))

In [ ]:
tfa_model.reactions.get_by_id("ATPS4r").lower_bound = 0

In [ ]:
solution = tfa_model.optimize()

In [ ]:
solution["BIOMASS_Ecoli_core_w_GAM"]

In [ ]:
cobra_model.slim_optimize()

In [ ]:
def build_core_model():
    model = load_model("e_coli_core")
    pH_I_T_dict = {
        "pH": {"c": 7.5, "e": 7, "p": 7},
        "I": {"c": 0.25, "e": 0, "p": 0},
        "T": {"c": 298.15, "e": 298.15, "p": 298.15},
    }
    del_psi_dict = {
        "c": {"c": 0, "e": 0, "p": 150},
        "e": {"c": 0, "e": 0, "p": 0},
        "p": {"c": -150, "e": 0, "p": 0},
    }
    del_psi = pd.DataFrame.from_dict(data=del_psi_dict)
    comp_info = pd.DataFrame.from_dict(data=pH_I_T_dict)

    excluded_reactions = [rxn.id for rxn in model.boundary] + [
        "BIOMASS_Ecoli_core_w_GAM",
        "O2t",
        "H2Ot",
    ]

    tfa_model = tmodel(
        model,
        Exclude_list=excluded_reactions,
        compartment_info=comp_info,
        membrane_potential=del_psi,
    )
    for met in tfa_model.metabolites:
        met.Kegg_id = "bigg.metabolite:" + met.id[:-2]

    tfa_model.update()

    return tfa_model

In [ ]:
model = build_core_model()

In [ ]:
solution_box = model.optimize(solve_method="mip")
print(solution_box.objective_value)

In [ ]:
solution_miqc = model.optimize()  # In this case we are solving the MIQCP
print(solution_miqc.objective_value)

In [ ]:
from multitfa.analysis import variability, variability_legacy_cplex

In [ ]:
dg_vars = [
    rxn.delG_forward.name
    for rxn in model.reactions
    if rxn.id not in model.Exclude_reactions
]

In [ ]:
box_ranges = variability(model, dg_vars)

In [ ]:
ranges_miqc = variability_legacy_cplex(model, variable_list=dg_vars)

In [ ]:
np.sqrt(np.power((box_ranges.maximum - box_ranges.minimum), 2.0).sum())

In [ ]:
np.sqrt(np.power((ranges_miqc.maximum - ranges_miqc.minimum), 2.0).sum())

In [ ]:
mask = (ranges_miqc.maximum - ranges_miqc.minimum) > (
    box_ranges.maximum - box_ranges.minimum
)

In [ ]:
ranges_miqc.loc[mask].shape

In [ ]:
ranges_miqc.loc[~mask].shape